## 과제명 : 발화상황 분류 모델 만들기 

### 발화 상황이 label 되어 있는 text를 이용해 발화 상황을 분류하는 모델을 만드시오.
제출기한 : 7/24(목) MN 까지 업로드 

1. 입력 데이터는 **한국어 원문** 사용
2. 분류 label은 table에 명시된 **‘소분류’를 대상**으로 함
3. 데이터 **전처리, 모델 빌드, 학습, 최적화, 평가 과정**을 포함
4. **편향된 label의 문제를 해결**하기 위한 하나 이상의 노력을 명시

과제 제출 형식 : ipynb파일로 작성하여 업로드

In [77]:
import collections
import os
import pathlib
import re
import string
import sys
import tempfile
import time
import json
from datetime import datetime
# 라이브러리 설치 필요 없음 
import pandas as pd 
from tqdm import tqdm 
import numpy as np
import matplotlib.pyplot as plt

import torch 
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchtext
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator 

import urllib.request
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [78]:
tf.get_logger().setLevel('ERROR')
pwd = pathlib.Path.cwd()

In [79]:
df = pd.read_excel('/home/alpaco/sryang/개인프로젝트/[발화] 개인프로젝트/2_대화체.xlsx')[['대분류', '소분류', '원문', '번역문']]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   대분류     100000 non-null  object
 1   소분류     100000 non-null  object
 2   원문      100000 non-null  object
 3   번역문     100000 non-null  object
dtypes: object(4)
memory usage: 3.1+ MB


In [80]:
df.isnull().sum()
from sklearn.model_selection import train_test_split
import pandas as pd 

df['소분류'].unique()
df['소분류'].value_counts()

소분류
쇼핑              10500
부서별              9101
교통               6844
회의               6360
여행               5225
식당               5128
공항               5092
회사 내             4859
숙소               4283
음식점              4060
학교               4052
온라인 쇼핑           3960
병원               3212
미용실/스파/뷰티샵       2428
엔터테인먼트           2236
출장               2179
영업               1960
긴급 상황            1744
운동               1560
컨퍼런스             1524
연예               1484
관광               1468
가정               1332
전화               1288
약국               1204
회식                964
전화업무              908
회사                888
경기관람              560
인사                535
스포츠               440
숙박                400
금융                360
일상                304
취미/관심사            300
데이트               285
엔터테이먼트            232
병원/약국             200
미용실/스파/뷰티         180
호텔                 44
외식                 36
인사관리               36
미용                 32
무역                 20
관공서/행정업무보는상황       20
사내 일상대

In [81]:
# 유일한 값의 개수 확인
df['대분류'].unique()
df['소분류'].unique()
df['원문'].unique()
df['번역문'].unique()

array(["How is the market's reaction to the newly released product?",
       'The sales increase is faster than the previous product.',
       "Then, we'll have to call the manufacturer and increase the volume of orders.",
       ..., 'Yes, of course, you just need to enter your phone number.',
       'I entered it, I want to pay it with all the department store point.',
       "I'm sorry, but you need to make a separate payment of 15,000 won, excluding the department store points."],
      dtype=object)

### 1. 카테고리화 및 인코딩 
### 2. Train, Valid, Test data 분리 

In [82]:
from sklearn.preprocessing import LabelEncoder
from pandas import json_normalize

# 카테고리화 및 인코딩 
def categorize_fields(df):
    records = []
    for _, row in df.iterrows():
        record={'소분류': row['소분류'],
                 '원문': row['원문'],
                 '번역문': row['번역문']}
        records.append(record)
    return pd.DataFrame(records)
structured_df = categorize_fields(df)    

print("분류 후 데이터 크기: ", structured_df.shape) 
print("분류 후 데이터 샘플:")
print(structured_df.head())

분류 후 데이터 크기:  (100000, 3)
분류 후 데이터 샘플:
  소분류                            원문  \
0  회의   이번 신제품 출시에 대한 시장의 반응은 어떤가요?   
1  회의    판매량이 지난번 제품보다 빠르게 늘고 있습니다.   
2  회의  그렇다면 공장에 연락해서 주문량을 더 늘려야겠네요.   
3  회의   네, 제가 연락해서 주문량을 2배로 늘리겠습니다.   
4  회의   지난 회의 마지막에 논의했던 안건을 다시 볼까요?   

                                                 번역문  
0  How is the market's reaction to the newly rele...  
1  The sales increase is faster than the previous...  
2  Then, we'll have to call the manufacturer and ...  
3  Sure, I'll make a call and double the volume o...  
4  Shall we take a look at the issues we discusse...  


In [83]:
def label_encoding(df, idx):
    label_map = {label: idx for idx, label in enumerate(labels)}
    encoded_data = [label_map[item] for item in df if item in label_map]   
    return encoded_data 

print("소분류 레이블 인코딩:")
labels = structured_df['소분류'].unique()
encoded_labels = label_encoding(structured_df['소분류'], labels)
print(encoded_labels)

소분류 레이블 인코딩:
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [84]:
y_data = encoded_labels

# 샘플이 1개 이하인 클래스 제거
df_filtered = df.groupby('소분류').filter(lambda x: len(x) > 1)
print(df_filtered['소분류'].value_counts())

소분류
쇼핑              10500
부서별              9101
교통               6844
회의               6360
여행               5225
식당               5128
공항               5092
회사 내             4859
숙소               4283
음식점              4060
학교               4052
온라인 쇼핑           3960
병원               3212
미용실/스파/뷰티샵       2428
엔터테인먼트           2236
출장               2179
영업               1960
긴급 상황            1744
운동               1560
컨퍼런스             1524
연예               1484
관광               1468
가정               1332
전화               1288
약국               1204
회식                964
전화업무              908
회사                888
경기관람              560
인사                535
스포츠               440
숙박                400
금융                360
일상                304
취미/관심사            300
데이트               285
엔터테이먼트            232
병원/약국             200
미용실/스파/뷰티         180
호텔                 44
외식                 36
인사관리               36
미용                 32
무역                 20
사내 일상대화            20
관공서/행정

In [85]:
# train & test 데이터 분리 
training, test_df = train_test_split(df_filtered, test_size=0.2, random_state=42, stratify=df_filtered['소분류'])
training

# train를 train & valid로 데이터 분리 
training_df, valid_df = int(len(training) * 0.8), int(len(training)*0.2)

In [86]:
print(training_df) 
print(valid_df)
print(len(test_df))
# training & valid_df 분리하는 과정에서 1개 손실 
print(len(df_filtered))

63999
15999
20000
99999


In [87]:
# 열마다의 NULL 값의 개수 확인
df_filtered.isnull().sum()

대분류    0
소분류    0
원문     0
번역문    0
dtype: int64

### 3. 데이터 전처리
    (1) 개체명 태깅
    (2) 중복, 결측치 확인 
    (3) 불용어, 특수기호 제거 
    (4) 토큰화
    (5) 벡터화  

In [106]:
from konlpy.tag import Okt # pip install konlpy
from konlpy.tag import Hannanum, Okt, Kkma
from transformers import BertTokenizer
from collections import Counter

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [97]:
print('소분류_tag 열의 값 개수 카운트')
print(df_filtered.groupby('소분류').size().reset_index(name='개수'))

소분류_tag 열의 값 개수 카운트
             소분류     개수
0        CS/고객상담     16
1         IT/컴퓨터     20
2             가정   1332
3           경기관람    560
4          경영/사무     20
5             공항   5092
6   관공서/행정업무보는상황     20
7             관광   1468
8             교통   6844
9             금융    360
10         긴급 상황   1744
11           데이트    285
12        마케팅/홍보     20
13            무역     20
14            미용     32
15     미용실/스파/뷰티    180
16    미용실/스파/뷰티샵   2428
17            법률     20
18            병원   3212
19         병원/약국    200
20           부서별   9101
21       사내 일상대화     20
22            쇼핑  10500
23            숙박    400
24            숙소   4283
25           스포츠    440
26            식당   5128
27          신발가게      4
28            약국   1204
29        엔터테이먼트    232
30        엔터테인먼트   2236
31            여행   5225
32           연구실      4
33            연예   1484
34            영업   1960
35        온라인 쇼핑   3960
36            외식     36
37            운동   1560
38           음식점   4060
39            인사    

In [98]:
data = df_filtered.fillna(method="ffill")

print(data.tail()) 

         대분류 소분류                                        원문  \
99995  여행/쇼핑  쇼핑        저희가 가격표 배치를 잘못해서 혼동을 드렸나 봐요, 죄송해요.   
99996  여행/쇼핑  쇼핑                 백화점 포인트로 계산하고 싶은데, 가능한가요?   
99997  여행/쇼핑  쇼핑                 네, 물론이죠, 전화번호 입력해주시면 됩니다.   
99998  여행/쇼핑  쇼핑              입력했어요, 전액 백화점 포인트로 결제하고 싶어요.   
99999  여행/쇼핑  쇼핑  죄송하지만 포인트 제외한 차액 15,000원은 따로 결제해주셔야 합니다.   

                                                     번역문  
99995  It seems that we didn't place the price tags c...  
99996       Can I pay using the department store points?  
99997  Yes, of course, you just need to enter your ph...  
99998  I entered it, I want to pay it with all the de...  
99999  I'm sorry, but you need to make a separate pay...  


In [99]:
# 전체 데이터에서 NULL 값 확인 
'NULL 값 확인 : ' + str(data.isnull().values.any())

'NULL 값 확인 : False'

In [108]:
tf.keras.layers.TextVectorization(
    max_tokens=None,
    standardize='lower_and_strip_punctuation',
    split='whitespace',
    ngrams=None,
    output_mode='int',
    output_sequence_length=None,
    pad_to_max_tokens=False,
    vocabulary=None,
    sparse=False,
)
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

In [109]:
# 토큰화
def preprocess_src_in(tokens):

    tokens = tokenizer.tokenize((tokens))
    return tokens

    review_text = re.sub("[^가-힣\s]", "", review)
    word_review = okt.pos(review_text, stem=True)

def preprocess_trg_in(tokens):
    tokens = tokenizer.tokenize((tokens))
    return tokens

def preprocess_trg_out(tokens):
    tokens = tokenizer.tokenize((tokens))
    return tokens

In [110]:
df['src_in'] = df['원문'].apply(lambda x : preprocess_src_in(x))
df['trg_in'] = df['번역문'].apply(lambda x : preprocess_trg_in(x))
df['trg_out'] = df['번역문'].apply(lambda x : preprocess_trg_out(x))

In [113]:
okt = Okt()

JVMNotFoundException: No JVM shared library file (libjvm.so) found. Try setting up the JAVA_HOME environment variable properly.

In [111]:
#노이즈/불용어 제거
stop_words = ['도','는','가','의','이','은','는','가','한','에','하','고','을','를','인','듯','과','와','네','들','듯','지','임','에','게']
def preprocess_tokens(tokens):
    word_review = [(token, pos) for token, pos in word_review if not token in stop_words]

    word_review = [(token, pos) for token, pos in word_review if len(token) > 1]

    return word_review

In [134]:
from konlpy.tag import Okt
from torchtext.data.utils import get_tokenizer
# 품사태깅 
okt = Okt()

def konlpy_tokenizer(text):
    return okt.morphs(text)  

# 길이계산
text_len = []
for label, text in train_raw_data:
    tokens = konlpy_tokenizer(text)
    text_len.append(len(tokens))

print(text_len)

JVMNotFoundException: No JVM shared library file (libjvm.so) found. Try setting up the JAVA_HOME environment variable properly.

In [114]:
def build_vocab(sents):
    word_list = []

    for sent in sents:
        for word in sent:
            word_list.append(word)
    word_counts = Counter(word_list)
    vocab = sorted(word_counts, key=word_counts.get, reverse=True)

    word_to_index = {}
    word_to_index['<PAD>'] = 0
    word_to_index['<UNK>'] = 1


    for index, word in enumerate(vocab):
        if word != '<PAD>':
            word_to_index[word] = index + 2

    return word_to_index

In [118]:
src_tokenizer = get_tokenizer("Kopy")
src_tokenizer.fit_on_texts(df['소분류'])
tar_tokenizer = get_tokenizer("Kopy")
tar_tokenizer.fit_on_texts(df['소분류'])

ValueError: Requested tokenizer Kopy, valid choices are a callable that takes a single string as input, "revtok" for the revtok reversible tokenizer, "subword" for the revtok caps-aware tokenizer, "spacy" for the SpaCy English tokenizer, or "moses" for the NLTK port of the Moses tokenization script.

In [120]:
if sentences:
    src_in = df['src_in'].to_list()
    trg_in = df['trg_in'].to_list()
    trg_out = df['trg_out'].to_list()

    src_vocab = build_vocab(src_in)
    tar_vocab = build_vocab(trg_in + trg_out)

    src_vocab_size = len(src_vocab)
    tar_vocab_size = len(tar_vocab)
    print("한국어 단어 집합 크기 : {:d}, 영어단어 집합 크기 : {:d}".format(src_vocab_size, tar_vocab_size))
else:
    print("sentences 리스트가 비어 있습니다")

sentences 리스트가 비어 있습니다


In [123]:
df[src_vocab_size] = df[src_vocab].apply(len)
df[tar_vocab_size] = df[tar_vocab].apply(len)

NameError: name 'src_vocab' is not defined

In [124]:
index_to_src = {v: k for k, v in src_vocab.items()}
index_to_tar = {v: k for k, v in tar_vocab.items()}

index_to_src

NameError: name 'src_vocab' is not defined

In [125]:
df['src_in_len'] = df['src_in'].apply(len)
df['trg_in_len'] = df['input'].apply(len)

KeyError: 'input'

In [126]:
print(df['src_in_len'].describe())
print(df['trg_in_len'].describe())

fig, ax = plt.subplots(1, 2, figsize=(12, 5))
ax[0].hist(df['src_in_len'], bins=40)
ax[0].set_title('Source Len')
ax[1].hist(df['trg_in_len'], bins=40)
ax[1].set_title('Target Len')
plt.show()

count    100000.000000
mean         20.866550
std           4.837873
min           5.000000
25%          18.000000
50%          20.000000
75%          23.000000
max          98.000000
Name: src_in_len, dtype: float64


KeyError: 'trg_in_len'

In [128]:
df = df[df['src_in_len'] <= 30].reset_index(drop=True)

In [129]:
df.iloc[0]['src_in']

['이',
 '##번',
 '신',
 '##제',
 '##품',
 '출',
 '##시',
 '##에',
 '대한',
 '시',
 '##장의',
 '반',
 '##응',
 '##은',
 '어떤',
 '##가',
 '##요',
 '?']

In [130]:
tf.keras.layers.TextVectorization(
    max_tokens=None,
    standardize='lower_and_strip_punctuation',
    split='whitespace',
    ngrams=None,
    output_mode='int',
    output_sequence_length=None,
    pad_to_max_tokens=False,
    vocabulary=None,
    sparse=False
)

ValueError: Requested tokenizer Konlpy, valid choices are a callable that takes a single string as input, "revtok" for the revtok reversible tokenizer, "subword" for the revtok caps-aware tokenizer, "spacy" for the SpaCy English tokenizer, or "moses" for the NLTK port of the Moses tokenization script.

AttributeError: 'int' object has no attribute 'drop_duplicates'

In [ ]:
TEXT_vocab = build_vocab_from_iterator(yield_tokens(train), specials=['<unk>','<pad>'], min_freq=20, max_tokens=1000)
TEXT_vocab.set_default_index(TEXT_vocab['<unk>'])

print(train_data[0])

In [ ]:
# train & test 데이터 분리 
training, test_df = train_test_split(df_filtered, test_size=0.2, random_state=42, stratify=df_filtered['소분류'])
training

,대분류,소분류,원문,번역문
956,비즈니스,회의,우리 이어폰 매출 하락에 관해 얘기하고 싶어요. 우리가 어떻게 해야 할까요?,I want to talk about the decrease in sales of ...
12555,여행/쇼핑,숙소,"숙소 먼저 들리려고 했는데, 우선 쇼핑몰부터 가야겠네.",Let's go to the shopping mall first before goi...
12065,여행/쇼핑,쇼핑,바로 옆 코너에 있습니다.,It's in the section right next to you.
32962,비즈니스,영업,견적을 우선 받아보고 결정할 수 있을 것 같네요.,I'll need to have a look at the quote before I...
54894,여행/쇼핑,관광,직접 체험할 수 있는 관광을 하고 싶습니다.,I want something I can experience.
...,...,...,...,...
19381,여행/쇼핑,쇼핑,"알겠습니다, 국내산 삼겹살 2킬로 말씀이시죠?","Okay, you mean two kilos of homegrown pork belly?"
62132,비즈니스,회사 내,여기 이력서에 중국으로 교환학생 간 적이 있다고 적혀있네요?,It says on your resume that you've been to Chi...
80127,비즈니스,회의,그래도 말만 잘하는 지원자보다 실력은 확실해서 좋은 거 같아요.,But I think his skills are more certain that t...
12216,여행/쇼핑,쇼핑,여기 있는 책가방은 세일하나요?,Is this backpack here for sale?


In [ ]:
clean_review = []
clean_review_test = []

for review in tqdm(train['소분류']):
    if type(review) == str:
        clean_review.append(preprocess(review))
    else:
        clean_review.append([])

for review in tqdm(test['소분류']):
    if type(review) == str:
        clean_review_test.append(preprocess(review))
    else:
        clean_review_test.append([])

In [ ]:
import pickle

with open('clean_review.pkl', 'wb') as f:
    pickle.dump(clean_review, f)
with open('clean_review_test.pkl', 'wb') as f:
    pickle.dump(clean_review_test, f)

In [ ]:
from torch.utils.data import DataLoader, Dataset

In [ ]:
class TranslationDataset(Dataset):
    def __init__(self, dataframe):
        self.src_in = dataframe['src_in'].values
        self.input = dataframe['trg_in'].values
        self.output = dataframe['trg_out'].values

    def __len__(self):
        return len(self.src_in)

    def __getitem__(self, idx):
        src_seq = torch.tensor(self.src_in[idx], dtype=torch.long)
        trg_in_seq = torch.tensor(self.input[idx], dtype=torch.long)
        trg_out_seq = torch.tensor(self.output[idx], dtype=torch.long)

        return src_seq, trg_in_seq, trg_out_seq

In [ ]:
#GRU 모듈 데이터 전처리
X_tensor = torch.FloatTensor(X).unsqueeze(2)
y_tensor = torch.FloatTensor(y).unsqueeze(1)

print(f"X tensor shape: {X_tensor.shape}")
print(f"y tensor shape: {y_tensor.shape}")

#모듈 파라미터
input_size =1
hidden_size = 32
num_layers=1
output_size=1
batch_size = len(X_tensor)

gru = nn.GRU(input_size=input_size,
             hidden_size=hidden_size,
             num_layers=num_layers,
             batch_first=True
             )
h0= torch.zeros(num_layers, batch_size, hidden_size)

gru_out, hn = gru(X_tensor, h0)

print(f"GRU 출력(out): {gru_out.shape}")
print(f"최종 hidden_state: {hn.shape}")

NameError: name 'X' is not defined

In [ ]:
class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(GRUModel, self).__init__()
        self.hidden_size=hidden_size
        self.num_layers = num_layers

        self.gru=nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out,_ = self.gru(x)

        out = self.fc(out[:,-1,:])

        return out

In [ ]:
#모델 학습

def train_model(model, train_loader, criterion, optimizer, num_epochs=100):
    model.train()
    losses = []

    for epoch in range(num_epchs):
        epoch_loss = 0
        for X_batch, y_batch in train_loader:
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)

            optimizer.zero_grad()
            loss.backword()
            optimizer.step()

            gru_model = GRUModel(input_size, hidden_size, num_layers, output_size)
            optimizer = optim.Adam(gru_model.parameters(), lr=0.001)

            print("\n==== GRU모델 학습 시작 ====")
            gru_losses = train_model(gru_model, train_loader, criterion, optimizer, num_epochs=100)
            gru_predictions, tru_actuals = evaluate_model(gru_model, test_loader, criterion)

            epoch_loss += loss.item()

        avg_loss = epoch_loss / len(train_loader)
        losses.append(avg_loss)

        if (epoch+1)%10 == 0:
            print(f"epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")
    return losses

3-5. 평가
- 지표 설정 및 측정
- 클래스 별 ROC AUC 시각화 

In [ ]:
#모델 평가
def evaluate_model(model, test_loader, criterion):
    model.eval()
    test_loss=0
    predictions =[]
    actuals =[]

    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            test_loss += loss.item()

            predictions.extend(outputs.numpy())
            actuals.extend(y_batch.numpy())
    avg_test_loss=test_loss / len(test_loader)
    rmse = np.sqrt(mean_squared_error(actuals, predictons))

    print(f'test loss: {avg_test_loss:.4f}, RMSE: {rmse:.4f}')
    return predictions, actuals

In [ ]:
def main():

    data, _ = generate_time_series(1000)

    scaler = MinMaxScaler(feature_range=(-1, 1))
    data_normalized = scaler.fit_transform(data.reshape(-1,1)).flatten()

    seq_length =10
    X, y = create_sequences(data_normalized, seq_length)

    #데이터 분할
    train_size=int(len(X)*0.8)
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]

    #텐서변환
    X_train = torch.FloatTensor(X_train).unsqueeze(2)
    X_test = torch.FloatTensor(X_test).unsqueeze(2)
    y_train = torch.FloatTensor(y_train).unsqueeze(1)
    y_test = torch.FloatTensor(y_test).unsqueeze(1)

    #데이터 로더 생성
    batch_size=32
    train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
    test_dataset = torch.utils.data.TensorDataset(X_test, y_test)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    #Model hypermarameter
    input_size =1
    hiddensize=32
    num_layers=2
    output_size=1

In [ ]:
#시각화
    plt.figure(figsize=(12,5))
    plt.subplot(1,2,1)

    plt.plot(losses, label='GRU')
    plt.xlabel('Epoch')

    plt.ylabel('Loss')
    plt.title('Training Loss')
    plt.legend()

    plt.subplot(1,2,2)
    plt.plot(gru_predictions, label='GRU Prediction')
    plt.xlabel('Time Step')
    plt.ylabel('Value')
    plt.title('Prediction vs. Actual')
    plt.legend()

    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    main()

4. 편향화된 label의 문제를 해결하기 위한 하나 이상의 노력